#National school of sciences and advanced technologies of Borj Cedria
###Deep learning tutorial 3: Text Classification using Neural Networks


Author: **MAAROUFI Sirine 2 TA 1**

# Importing Required Packages

In [ ]:
import numpy as np
import pandas as pd
import string
from keras.models import Model
from keras import Input


In [ ]:
from keras.layers import *
import text_cleaners as txt
from sklearn.feature_extraction.text import TfidfVectorizer

# Load Data

In [ ]:
 dframe = pd.read_csv("news.csv", sep=',', encoding = "ISO-8859-1",
                    error_bad_lines=False)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
len(dframe)

2225

In [ ]:
dframe[:20]

,news,type
0,China had role in Yukos split-up\n \n China le...,business
1,Oil rebounds from weather effect\n \n Oil pric...,business
2,Indonesia 'declines debt freeze'\n \n Indonesi...,business
3,$1m payoff for former Shell boss\n \n Shell is...,business
4,US bank in $515m SEC settlement\n \n Five Bank...,business
5,Verizon 'seals takeover of MCI'\n \n Verizon h...,business
6,"Parmalat boasts doubled profits\n \n Parmalat,...",business
7,US seeks new $280bn smoker ruling\n \n The US ...,business
8,"Steel firm 'to cut' 45,000 jobs\n \n Mittal St...",business
9,Cars pull down US retail figures\n \n US retai...,business


In [ ]:
#list_of_words = dframe['Word'].values.tolist() ; print(list_of_words[:10])
news = dframe['news'].values.tolist()
types = dframe['type'].values.tolist()
print(news[1])
print(set(types))

Oil rebounds from weather effect
 
 Oil prices recovered in Asian trade on Tuesday, after falling in New York on milder winter weather across the US.
 
 With winter temperatures staying relatively high in the northern US, a barrel of light crude ended Monday down $1.33 to $42.12. However crude prices have rebounded in Asia, rising to $42.30 a barrel for February delivery. In London, trading of Brent crude was suspended for a public holiday, but the price fell to $39.20 in the Far East.
 
 With milder temperatures expected to continue in the northern parts of the US over the next few days at least, analysts have said the price of oil may fall further - even if the decline was only temporary. "Weather has been the Achilles' heel of this market," said ABN AMRO analyst John Brady. "But it is winter in the northeast. Eventually we'll get another cold blast." Despite a fall of more than $12 a barrel from the record highs reached in late October, the price of crude oil remains almost 30% high

In [ ]:

from nltk import sent_tokenize

# Preprocessing Step and Text Cleaning


In [ ]:
# Remove stop words
custom_stop_words = ['a','an','the','this','that','am','is','are','has','had','was','were']
cleaned_sent = txt.remove_stop(news,custom_stop_words)
cleaned_sent[5]

"Verizon 'seals takeover of MCI' Verizon won takeover battle for US phone firm MCI with bid worth $6.8bn (Â£3.6bn), reports say. two firms expected to seal deal on Monday morning, according to news agency reports, despite what thought to be higher bid from Qwest. US telecoms market consolidating fast, with former long-distance giant AT&T being bought by former subsidiary SBC earlier year for $16bn. MCI exited bankruptcy in April, having gone bust under previous name WorldCom. bankruptcy followed its admission in 2002 it illegally booked expenses and inflated profits. Shareholders lost about $180bn when company collapsed, while 20,000 workers lost their jobs. Former Worldcom boss Bernie Ebbers currently on trial, accused of overseeing $11bn fraud. Qwest itself come under suspicion of sub-standard behaviour, paying Securities and Exchange Commission $250m in October to settle charges it manipulated its results to keep Wall Street happy. MCI US's second-biggest long distance firm after AT

In [ ]:
news[5]

"Verizon 'seals takeover of MCI'\n \n Verizon has won a takeover battle for US phone firm MCI with a bid worth $6.8bn (Â£3.6bn), reports say.\n \n The two firms are expected to seal the deal on Monday morning, according to news agency reports, despite what was thought to be a higher bid from Qwest. The US telecoms market is consolidating fast, with former long-distance giant AT&T being bought by former subsidiary SBC earlier this year for $16bn. MCI exited bankruptcy in April, having gone bust under previous name WorldCom. The bankruptcy followed its admission in 2002 that it illegally booked expenses and inflated profits.\n \n Shareholders lost about $180bn when the company collapsed, while 20,000 workers lost their jobs. Former Worldcom boss Bernie Ebbers is currently on trial, accused of overseeing an $11bn fraud. Qwest has itself come under suspicion of sub-standard behaviour, paying the Securities and Exchange Commission $250m in October to settle charges that it manipulated its r

In [ ]:
# list of ponctuation that will be removed from texts
print(list(string.punctuation))
pnt = list(string.punctuation)
pnt.remove(':') # we kept the ':' in this example
Sent_without_pont = txt.removePonct(cleaned_sent,pnt = pnt)

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']


In [ ]:
# The list of categories
TYPES = list(set(types))
print(TYPES)

['sport', 'politics', 'business', 'entertainment', 'tech']


### One hot encoding
One hot encoding is a process of converting categorical data variablesso they can be provided to machine learning algorithms to improve predictions.

In [ ]:
y = []
for i in range(len(types)):
 if(types[i] == TYPES[0]):
  y.append([1,0,0,0,0])
 elif(types[i] == TYPES[1]):
  y.append([0,1,0,0,0])
 elif(types[i] == TYPES[2]):
  y.append([0,0,1,0,0])
 elif(types[i] == TYPES[3]):
  y.append([0,0,0,1,0])
 elif(types[i] == TYPES[4]):
  y.append([0,0,0,0,1])
Y = np.asarray(y)
print(Y.shape)

(2225, 5)


### TfidfVectorizer
TF-IDF is an abbreviation for Term Frequency Inverse Document Frequency. This is very common algorithm to transform text into a meaningful representation of numbers which is used to fit machine algorithm for prediction.
In simple words, TFIDF is a numerical statistic that shows the importance of a word in a text document.

In [ ]:
# Define the vectoriser
vectorizer = TfidfVectorizer(min_df=1, max_df=0.9)

In [ ]:
# Split the data to training and testing sets
from sklearn.model_selection import train_test_split
sent_train, sent_test, y_train, y_test = train_test_split(Sent_without_pont, Y, test_size=0.3)

In [ ]:
# Fit the vectoriser to the data
X_train = vectorizer.fit_transform(sent_train) # from documenets to tf-idf representation for training.
X_test = vectorizer.transform(sent_test)

In [ ]:
print(X_train.shape)

(1557, 29179)


In [ ]:
print(X_test.shape)

(668, 29179)


In [ ]:
num_labels = len(TYPES)
vocab_size = X_train.shape[1]

In [ ]:
print(vocab_size)

29179


#  Neural network model

### Logistic Regression Model


In [ ]:
# Define the logistic regression model
inp1 = Input(shape = (vocab_size,))
x = Dense(num_labels,activation = 'softmax')(inp1)
lr = Model(inp1,x)

###  MLP with layers type

In [ ]:
# Define the MLP model
Inp = Input(shape = (vocab_size,))
x1 = Dense(units = 100,activation = 'relu')(Inp)
x = Dense(num_labels,activation = 'softmax')(x1)
mlp = Model(Inp, x)

In [ ]:
# Compile using mean_squared error as a loss function
mlp.compile(optimizer="adamax", loss="mean_squared_error", metrics=["accuracy"])

In [ ]:
# Fit the model with validation_split = 0.1
history = mlp.fit(X_train.toarray(), y_train ,
                  validation_data = (X_test.toarray(),y_test),
                  batch_size=25, epochs=60 , verbose=1)

Epoch 1/60
63/63 [==============================] - 3s 32ms/step - loss: 0.1483 - accuracy: 0.7938 - val_loss: 0.1319 - val_accuracy: 0.9446
Epoch 2/60
63/63 [==============================] - 2s 28ms/step - loss: 0.1129 - accuracy: 0.9769 - val_loss: 0.0962 - val_accuracy: 0.9521
Epoch 3/60
63/63 [==============================] - 2s 30ms/step - loss: 0.0738 - accuracy: 0.9865 - val_loss: 0.0640 - val_accuracy: 0.9716
Epoch 4/60
63/63 [==============================] - 2s 36ms/step - loss: 0.0441 - accuracy: 0.9923 - val_loss: 0.0435 - val_accuracy: 0.9775
Epoch 5/60
63/63 [==============================] - 2s 33ms/step - loss: 0.0269 - accuracy: 0.9949 - val_loss: 0.0320 - val_accuracy: 0.9775
Epoch 6/60
63/63 [==============================] - 3s 42ms/step - loss: 0.0175 - accuracy: 0.9961 - val_loss: 0.0254 - val_accuracy: 0.9790
Epoch 7/60
63/63 [==============================] - 2s 39ms/step - loss: 0.0120 - accuracy: 0.9968 - val_loss: 0.0212 - val_accuracy: 0.9790
Epoch 8/60
63

## Model Predictions

In [ ]:
pred = mlp.predict(X_test.toarray())
np.argmax(pred[4])
pred_classes = [TYPES[np.argmax(x)] for x in pred]
orig = [TYPES[np.argmax(x)] for x in y_test]
print("Predicted values : ",pred_classes[0:100])
print("Original values : ",orig[0:100])

21/21 [==============================] - 0s 13ms/step
Predicted values :  ['tech', 'entertainment', 'business', 'business', 'business', 'sport', 'politics', 'sport', 'sport', 'tech', 'sport', 'business', 'business', 'sport', 'business', 'tech', 'business', 'business', 'entertainment', 'business', 'entertainment', 'tech', 'politics', 'entertainment', 'sport', 'business', 'politics', 'sport', 'politics', 'sport', 'politics', 'sport', 'sport', 'entertainment', 'business', 'entertainment', 'tech', 'sport', 'politics', 'tech', 'politics', 'politics', 'politics', 'sport', 'tech', 'entertainment', 'tech', 'entertainment', 'business', 'sport', 'sport', 'politics', 'politics', 'business', 'sport', 'tech', 'tech', 'tech', 'politics', 'tech', 'business', 'entertainment', 'sport', 'tech', 'sport', 'entertainment', 'entertainment', 'sport', 'politics', 'tech', 'politics', 'tech', 'sport', 'entertainment', 'tech', 'sport', 'politics', 'tech', 'sport', 'business', 'business', 'entertainment', 'sport'

In [ ]:
# captures indexes of misclassified numbers
ERR = []
for i in range(y_test.shape[0]):
    if(pred_classes[i] != orig[i]):
        ERR.append(i)

In [ ]:
len(ERR)

522

# Conclusion
**Multilayer perceptrons are a powerful tool in the field of machine learning. This algorithm fitted on the news data gave an accuracy of 1 and a validation accuracy of 0.9865.**